# 3. World Graph APIs

你可以参考 [Partnr Repository - World Graph](https://github.com/RoboticSJTU/partnr-planner/blob/habitat_llm/world_model/dynamic_world_graph.py) 来了解在Task Planning过程中接触到的World Graph具体有哪些属性。

本文档则将以一个简单的场景为例，介绍这个World Graph的一些API 接口。

In [1]:
from omegaconf.dictconfig import DictConfig
from embodiment.dummy_planner import DummyPlanner

from python_package.embodiment.runner import PartnerRunner
from hydra import initialize, compose

initialize(config_path="conf")
# 通过 overrides 来覆盖配置文件中的参数
# 为了演示完整的 WorldGraph，这里将 world_model.partial_obs 设置为 False，
# 注意测评时 world_model.partial_obs=True
config = compose(config_name="example_simple_planner.yaml",
                 overrides=[
                    "world_model.partial_obs=False",
                    "evaluation.output_dir=outputs", "paths.results_dir=outputs"])
runner = PartnerRunner(config)

/tmp/ipykernel_4074643/46535709.py:7: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="conf")
/home/yongxi/App/anaconda3/envs/habitat-llm/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'example_simple_planner.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
2025-05-22 21:47:59,777 initializing sim CollaborationSim-v0


Finished setting up config


[21:48:02:340981]:[Error]:[URDF] URDFParser.cpp(680)::parseGeometry : Collision box primitive with 0 scale detected  Vector(0.3706, 0, 0.522) . Replacing zeros with 0.001 and continuing.
2025-05-22 21:48:02,673 Initializing task RearrangeEmptyTask-v0
MeshTools::compile(): ignoring unknown/unsupported attribute Trade::MeshAttribute::Custom(0)
MeshTools::compile(): ignoring unknown/unsupported attribute Trade::MeshAttribute::Custom(1)


successfully added agent with UID : 0
finished initializing agents!


## 3.1 Node
    
`habitat_llm.world_model.entity.Entity` 是所有 Node 所共同继承的基类，其定义了Node的各类属性。我们从World Graph 中取出2个节点观察一下。

In [2]:
from habitat_llm.world_model.world_graph import WorldGraph
world_graph: WorldGraph = runner.env_interface.world_graph[0]
object = world_graph.get_all_objects()[0]
print(type(object))
print(object.name)
print(object.properties)

<class 'habitat_llm.world_model.entity.Object'>
plant_container_0
{'type': 'plant_container', 'translation': [-7.822080135345459, 0.6565099954605103, 0.041999999433755875], 'states': {'is_clean': False, 'is_filled': False}}


在上面的节点中：
1. ```obj.name```是我们执行操作时需要的物体名称
2. ```obj.properties['type']```是这个物体的类型
3. ```obj.properties['translation']```是这个物体在空间中的位置，本题目中不需要关注
4. ```obj.properties['states']```是这个物体可被操作的属性，比如有```'is_clean'```的话表示可以对它使用清洁技能

In [3]:
furniture = world_graph.get_all_furnitures()[20]
print(type(furniture))
print(furniture.name)
print(furniture.properties)

<class 'habitat_llm.world_model.entities.furniture.Furniture'>
chair_20
{'type': 'chair', 'is_articulated': False, 'translation': Vector(-12.201, 0.460715, -2.0428), 'components': [], 'states': {'is_clean': False}}


在上面的节点中：
1. furniture.name是我们执行操作时需要的家具名称
2. ```['type']```和```['translation']```与上面的object一样。
3. ```obj.properties['is_articulated']```表示这个物体是否带关节，也就是可以用Open/Close技能操作它的意思。

## 3.2 Graph

WorldGraph 本身是一个图数据结构，`habitat_llm.world_model.graph.Graph` 是所有世界图所共同继承的基类，其定义了图的各类属性。我们用上面的World Graph中观察一下。

In [48]:
from habitat_llm.world_model.entity import Room, Receptacle
print(world_graph.get_neighbors(object))
print(world_graph.get_neighbors(furniture))
print(world_graph.get_neighbors_of_type(furniture, Receptacle))
print(world_graph.get_neighbors_of_type(furniture, Room))

{<habitat_llm.world_model.entity.Receptacle object at 0x7fe49c7f9700>: 'on'}
{<habitat_llm.world_model.entity.Room object at 0x7fe49c5cf460>: 'inside', <habitat_llm.world_model.entity.Receptacle object at 0x7fe49c7e7760>: 'joint', <habitat_llm.world_model.entity.Receptacle object at 0x7fe49c7e77c0>: 'joint'}
[<habitat_llm.world_model.entity.Receptacle object at 0x7fe49c7e7760>, <habitat_llm.world_model.entity.Receptacle object at 0x7fe49c7e77c0>]


`get_neighbors`会返回一个以 Node 为 key, relation 为 value 的 Dict 对象，代表所有与该节点相邻的节点。

`get_neighbors_of_type` 则会返回一个节点列表，代表特定类型的邻节点。

In [49]:
print("Rooms: ", world_graph.get_all_rooms())
print("Objects: ", world_graph.get_all_objects())
print("Furnitures: ", world_graph.get_all_furnitures())
print("Regions: ", world_graph.get_all_rooms())

Rooms:  [<habitat_llm.world_model.entity.Room object at 0x7fe49c5cf340>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf400>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf460>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf4c0>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf520>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf580>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf5e0>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf640>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf6a0>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf700>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf760>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf7c0>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf820>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf880>, <habitat_llm.world_model.entity.Room object at 0x7fe49c5cf8e0>]
Objects:  [<habitat_llm.world_m

上面的 `get_all_{NODE_TYPE}`函数可以返回世界图里面对应类的 Node List。

WorldGraph 可能包含以下几类 Node

In [ ]:
from habitat_llm.world_model.entity import (
    House,          # 根节点
    Room,           # 房间
    Receptacle,     # 可摆放区域
    Object,         # 物体
    SpotRobot,      # Agent
    Human,          # 人类，本项目不牵扯人机协作，可以忽略
)

from habitat_llm.world_model.entities.furniture import Furniture # 家具
from habitat_llm.world_model.entities.floor import Floor        # 地板，继承自 Furniture，可以理解为特殊的家具

除了直接获取完整列表，WorldGraph 还提供了按照属性检索节点的接口，例如

In [27]:
# 按照节点名称检索
node_from_name = world_graph.get_node_from_name("agent_0")
print(node_from_name.name, node_from_name)

# 按照节点属性检索
# 注意，get_node_with_property 会返回第一个匹配的节点
node_from_properties = world_graph.get_node_with_property(property_key='is_articulated', property_val=False)
print(node_from_properties.name, node_from_properties)
node_from_properties = world_graph.get_node_with_property(property_key='type', property_val='table')
print(node_from_properties.name, node_from_properties)

agent_0 SpotRobot[name=agent_0, type=entity_node]
chair_14 Furniture[name=chair_14, type=chair]
table_21 Furniture[name=table_21, type=table]


WorldGraph 还提供了一些特殊的判断节点状态的接口，例如

In [28]:
print(world_graph.is_object_with_robot(object))

False


上面的`is_object_with_robot`会返回物体是否在机器人上。

In [29]:
print(world_graph.find_object_furniture_pairs())

{<habitat_llm.world_model.entity.Object object at 0x7fe49c7f9760>: <habitat_llm.world_model.entities.furniture.Furniture object at 0x7fe49c5c64f0>, <habitat_llm.world_model.entity.Object object at 0x7fe49c7bb0a0>: <habitat_llm.world_model.entities.furniture.Furniture object at 0x7fe49c5c67f0>, <habitat_llm.world_model.entity.Object object at 0x7fe49c7d7f40>: <habitat_llm.world_model.entities.furniture.Furniture object at 0x7fe49c5c6790>, <habitat_llm.world_model.entity.Object object at 0x7fe49c7f9820>: <habitat_llm.world_model.entities.furniture.Furniture object at 0x7fe49c5bd7f0>, <habitat_llm.world_model.entity.Object object at 0x7fe49c7f9b20>: <habitat_llm.world_model.entities.furniture.Furniture object at 0x7fe49c5c6910>, <habitat_llm.world_model.entity.Object object at 0x7fe49c7d78e0>: <habitat_llm.world_model.entities.furniture.Furniture object at 0x7fe49c5cfa90>, <habitat_llm.world_model.entity.Object object at 0x7fe49c7d9820>: <habitat_llm.world_model.entities.furniture.Furnitu

`find_object_furniture_pairs`会返回世界图里面`Object`和`Furniture`的键值对

In [30]:
print(world_graph.get_subgraph([object]).to_string())

Room: living_room_1
	Furniture: table_30
		Receptacle: rec_table_30_0
			Object: plant_container_0



`get_subgraph` 返回给定Node在Agent可见范围内的到房间节点的子图。

In [31]:
for furniture in world_graph.get_all_furnitures():
    if "faucet" in furniture.properties.get("components", []):
        print(furniture.name)

shower_29
bathtub_44
shower_57
cabinet_88
cabinet_89


可以用`properties.get` 来检查哪些地方有水龙头，用于fill/pour的操作。

In [32]:
print(world_graph.to_string())

Room: bathroom_1
	Furniture: bathtub_44
		Receptacle: rec_bathtub_44_0
	Furniture: floor_bathroom_1
	Furniture: toilet_60
		Receptacle: rec_toilet_60_0
		Receptacle: rec_toilet_60_1
Room: bathroom_2
	Furniture: floor_bathroom_2
	Furniture: shower_29
		Receptacle: rec_shower_29_0
	Furniture: toilet_58
		Receptacle: rec_toilet_58_0
		Receptacle: rec_toilet_58_1
Room: bathroom_3
	Furniture: floor_bathroom_3
	Furniture: shower_57
		Receptacle: rec_shower_57_0
	Furniture: toilet_59
		Receptacle: rec_toilet_59_0
		Receptacle: rec_toilet_59_1
	Furniture: unknown_87
		Receptacle: rec_unknown_87_0
		Receptacle: rec_unknown_87_1
		Receptacle: rec_unknown_87_2
		Receptacle: rec_unknown_87_3
		Receptacle: rec_unknown_87_4
		Receptacle: rec_unknown_87_5
Room: bedroom_1
	Furniture: bed_38
		Receptacle: rec_bed_38_0
		Receptacle: rec_bed_38_1
		Receptacle: rec_bed_38_2
		Receptacle: rec_bed_38_3
			Object: egg_5
		Receptacle: rec_bed_38_4
		Receptacle: rec_bed_38_5
	Furniture: chair_55
		Receptacle: 

当然，查看World Graph最直接的方式是`to_string`。

你可以参考 [API Doc: WorldGraph](http://localhost:8000/api/WorldGraph/) 获得更详细的 WorldGraph API 文档。